In [12]:
import pandas as pd
import re
import itertools
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import gaussian_kde
import seaborn as sns
%matplotlib qt
from matplotlib.patches import Patch

df=pd.read_csv("milano_dati_booking.csv")
df['prezzo']= df['prezzo'].str.replace(".","")
df['prezzo'] = df['prezzo'].apply(lambda x: int(re.search(r'\d+', x).group()))
df['prezzo_iniziale']= df['prezzo_iniziale'].str.replace(".","")
df['prezzo_iniziale'] = df['prezzo_iniziale'].apply(lambda x: int(re.search(r'\d+', x).group()))


# Crea delle colonne con valori True/False in base alle offerte
df['booking_paga'] = df['offerte'].str.contains('Booking.com paga')
df['offerta_tempo_limitato'] = df['offerte'].str.contains('tempo limitato')
df['offerta_solo_mobile'] = df['offerte'].str.contains('mobile')
df['offerta_super_segreta'] = df['offerte'].str.contains('SuperSegreta')
df['offerta_inizio_2024'] = df['offerte'].str.contains('Offerta Inizio 2024')
# Inserisce False nei campi nulli
df[['booking_paga','offerta_tempo_limitato','offerta_solo_mobile','offerta_super_segreta','offerta_inizio_2024']] = df[['booking_paga','offerta_tempo_limitato','offerta_solo_mobile','offerta_super_segreta','offerta_inizio_2024']].fillna(False)

In [13]:
df1 = df[df["TIMESTAMP"]=="03-07-2024---16:09"]
df2 = df[df["TIMESTAMP"]=="03-07-2024---17:32"]
df3 = df[df["TIMESTAMP"]=="03-07-2024---14:59"]
df4 = df[df["TIMESTAMP"]=="03-07-2024---15:55"]

df1 = df1.drop_duplicates()
df2 = df2.drop_duplicates()
df3 = df3.drop_duplicates()
df4 = df4.drop_duplicates()

In [14]:
#Eseguo merge ma senza evidenziare differenze prezzo

merged_df = pd.merge(df1, df2, on=['nome_hotel','data','prezzo_iniziale'], how='inner')
# Elimino hotel con stesso nome
merged_df = merged_df.drop_duplicates(subset="nome_hotel",keep= False)  
merged_df[['nome_hotel','prezzo_x','prezzo_y','prezzo_iniziale']]

,nome_hotel,prezzo_x,prezzo_y,prezzo_iniziale
2,IMHOME - Porta Romana,89,98,108
3,Hotel Del Sud,50,56,60
4,Easy Milano - Rooms and Apartments Navigli,99,99,118
5,Hotel Aurelia Milano Centrale,98,108,119
6,Hotel Bolzano,114,125,149
...,...,...,...,...
677,CHALET,96,96,114
678,Charming Milan Apartments Brera - Madonnina,229,229,229
679,Hotel Mercure Milano Solari,161,161,161
680,Casa Venus IEO,92,92,99


In [15]:
merged_df2 = pd.merge(merged_df, df3, on=['nome_hotel','data','prezzo_iniziale'], how='inner')
# Elimino hotel con stesso nome
merged_df2 = merged_df2.drop_duplicates(subset="nome_hotel",keep= False)  
merged_df2[['nome_hotel','prezzo_x','prezzo_y','prezzo']]


,nome_hotel,prezzo_x,prezzo_y,prezzo
0,IMHOME - Porta Romana,89,98,98
1,Hotel Del Sud,50,56,60
2,Easy Milano - Rooms and Apartments Navigli,99,99,107
3,Hotel Aurelia Milano Centrale,98,108,108
6,[Superb Loft Navigli]Comfy&Relax,65,65,70
...,...,...,...,...
630,CHALET,96,96,104
631,Charming Milan Apartments Brera - Madonnina,229,229,229
632,Hotel Mercure Milano Solari,161,161,161
633,Casa Venus IEO,92,92,99


In [16]:
merged_df3 = pd.merge(merged_df2, df4, on=['nome_hotel','data','prezzo_iniziale'], how='inner',suffixes=("_w","_z"))
merged_df3.columns
merged_df3[['nome_hotel','prezzo_x','prezzo_y','prezzo_w','prezzo_z','username_x','username_y','username_w','username_z']]


,nome_hotel,prezzo_x,prezzo_y,prezzo_w,prezzo_z,username_x,username_y,username_w,username_z
0,IMHOME - Porta Romana,89,98,98,89,marcofantile,marcofantile,marcofantile,marcofantile
1,Hotel Del Sud,50,56,60,55,marcofantile,marcofantile,marcofantile,marcofantile
2,Easy Milano - Rooms and Apartments Navigli,99,99,107,107,marcofantile,marcofantile,marcofantile,marcofantile
3,Hotel Aurelia Milano Centrale,98,108,108,98,marcofantile,marcofantile,marcofantile,marcofantile
4,[Superb Loft Navigli]Comfy&Relax,65,65,70,70,marcofantile,marcofantile,marcofantile,marcofantile
...,...,...,...,...,...,...,...,...,...
549,Porta Venezia Home,98,98,106,106,marcofantile,marcofantile,marcofantile,marcofantile
550,Hotel Wagner,167,167,181,181,marcofantile,marcofantile,marcofantile,marcofantile
551,Charming Milan Apartments Brera - Madonnina,229,229,229,229,marcofantile,marcofantile,marcofantile,marcofantile
552,Hotel Mercure Milano Solari,161,161,161,161,marcofantile,marcofantile,marcofantile,marcofantile


In [17]:
merged_df3[['TIMESTAMP_x','TIMESTAMP_y','TIMESTAMP_w','TIMESTAMP_z','os_x','os_y','os_w','os_z']]
#merged_df3 ['diff_prezzo'] = merged_df3['prezzo_z'] - merged_df3['prezzo_x']

,TIMESTAMP_x,TIMESTAMP_y,TIMESTAMP_w,TIMESTAMP_z,os_x,os_y,os_w,os_z
0,03-07-2024---16:09,03-07-2024---17:32,03-07-2024---14:59,03-07-2024---15:55,Android,Linux,Linux,Android
1,03-07-2024---16:09,03-07-2024---17:32,03-07-2024---14:59,03-07-2024---15:55,Android,Linux,Linux,Android
2,03-07-2024---16:09,03-07-2024---17:32,03-07-2024---14:59,03-07-2024---15:55,Android,Linux,Linux,Android
3,03-07-2024---16:09,03-07-2024---17:32,03-07-2024---14:59,03-07-2024---15:55,Android,Linux,Linux,Android
4,03-07-2024---16:09,03-07-2024---17:32,03-07-2024---14:59,03-07-2024---15:55,Android,Linux,Linux,Android
...,...,...,...,...,...,...,...,...
549,03-07-2024---16:09,03-07-2024---17:32,03-07-2024---14:59,03-07-2024---15:55,Android,Linux,Linux,Android
550,03-07-2024---16:09,03-07-2024---17:32,03-07-2024---14:59,03-07-2024---15:55,Android,Linux,Linux,Android
551,03-07-2024---16:09,03-07-2024---17:32,03-07-2024---14:59,03-07-2024---15:55,Android,Linux,Linux,Android
552,03-07-2024---16:09,03-07-2024---17:32,03-07-2024---14:59,03-07-2024---15:55,Android,Linux,Linux,Android


In [28]:
# Grafico a dispersione che mostra i prezzi
plt.style.use("seaborn-v0_8-dark")

merged_df4 = merged_df3.sort_values(by="prezzo_y").reset_index(drop=True)
merged_df4 = merged_df4[merged_df4['prezzo_x']!=merged_df4['prezzo_y']]
merged_df4 = merged_df4[merged_df4['prezzo_w']!=merged_df4['prezzo_y']]
#merged_df4 = merged_df4[merged_df4['prezzo_z']!=merged_df4['prezzo_y']]



#plt.scatter(merged_df4.index,merged_df4['prezzo_z'], label='Ricerca Mobile Normale')
plt.scatter(merged_df4.index,merged_df4['prezzo_w'], label='Ricerca Desktop Normale')
plt.scatter(merged_df4.index,merged_df4['prezzo_y'], label='Ricerca Desktop Anomala')
plt.scatter(merged_df4.index,merged_df4['prezzo_x'], label='Ricerca Mobile Anomala')

plt.legend(fontsize=18)

plt.show()

In [29]:
# Grafico a linea che mostra i prezzi
plt.style.use("seaborn-v0_8-dark")

plt.scatter(merged_df4.index,merged_df4['prezzo_w'], label='Typical Desktop Search')
plt.scatter(merged_df4.index,merged_df4['prezzo_y'], label='Atypical Desktop Search')
plt.scatter(merged_df4.index,merged_df4['prezzo_x'], label='Atypical Mobile Search')
plt.xticks([]) 
plt.yticks([])
plt.xlabel('Hotels',fontsize=14)
plt.ylabel("Price",fontsize=14)

#plt.scatter(merged_df4.index,merged_df4['prezzo_z'], label='Ricerca Mobile Normale')


plt.legend(fontsize=18)

plt.show()

In [27]:
# Grafico che confronta i prezzi a parità di ricerca desktop 
merged_df4 = merged_df3.sort_values(by="prezzo_y").reset_index(drop=True)
merged_df4 = merged_df4[merged_df4['prezzo_w']!=merged_df4['prezzo_y']]
plt.style.use("seaborn-v0_8-dark")

plt.scatter(merged_df4.index, merged_df4['prezzo_w'], label='Ricerca Desktop Normale')
plt.scatter(merged_df4.index,merged_df4['prezzo_y'], label='Ricerca Desktop Anomala')
plt.xticks([]) 
plt.yticks([])
plt.legend()
plt.show()

In [21]:
# Grafico che confronta i prezzi a parità di ricerca mobile
merged_df4 = merged_df3.sort_values(by="prezzo_x").reset_index(drop=True)
merged_df4 = merged_df4[merged_df4['prezzo_x']!=merged_df4['prezzo_z']]
plt.style.use("seaborn-v0_8-dark")

plt.plot(merged_df4.index, merged_df4['prezzo_z'], label='Ricerca Mobile Normale')
plt.plot(merged_df4.index,merged_df4['prezzo_x'], label='Ricerca Mobile Anomala')
plt.xticks([]) 
plt.yticks([])
plt.legend()
plt.show()

In [22]:
# Grafico che confronta i prezzi tra ricerca desktop anomala e ricerca mobile 
merged_df4 = merged_df3.sort_values(by="prezzo_y").reset_index(drop=True)
merged_df4 = merged_df4[merged_df4['prezzo_y']!=merged_df4['prezzo_z']]
plt.style.use("seaborn-v0_8-dark")

plt.plot(merged_df4.index, merged_df4['prezzo_z'], label='Ricerca Mobile Normale')
plt.plot(merged_df4.index,merged_df4['prezzo_y'], label='Ricerca Desktop Anomala')
plt.xticks([]) 
plt.yticks([])
plt.legend()
plt.show()